# OpenGridMap Notebook  -- MST along road

Before executing any code in this Notebook, you should 
1. Install what specified in Installation.ipynb
2. Understand DataPreparation.ipynb is also helpful

The goal of this notebook is to
1. Learn APIs of NetworkX
2. Inference and test with different algorithms

# Step1. Data Preparation

## Import necessary modules

In [1]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from networkx.algorithms import approximation as approx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)
import math
import matplotlib.pyplot as plt
%matplotlib inline


def cleanMap(transJson, vornoiJson): 
    # Mark transformer
    tCenter = transJson['coordinates']
    tMarker =  Marker(location=tCenter)
    # Mark polygon
    tPoly = Polygon(locations=vornoiJson['coordinates'], weight=2,
                color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
                fill_color='#ccffcc') 
    
    tMap = Map(default_tiles=TileLayer(opacity=1.0),center=tCenter ,zoom=16)
    tMap.add_layer(tMarker)
    tMap.add_layer(tPoly)
    return tMap

def connect():# Set connection to gis db
    try:
        conn = psycopg2.connect("dbname=gis user=jennyzhou")
    except:
        print 'Fail to connect to Postgres Server'
    return conn

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Database connection

In [32]:
conn = connect()

## Select transformer, house within one vornoi area

In [39]:
cur = conn.cursor()
vid = 38
# Select vornoi polygon and transformer
cur.execute("SELECT ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(way, 4326)))), "
            "ST_AsGeoJSON((ST_FlipCoordinates(ST_Transform(tway, 4326))))FROM vornoi WHERE vid = %d ;" % vid)
data = cur.fetchall()

# Select all the houses
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) "
            "FROM vornoi_map where vid = %d and istransformer = false ;" % vid)
houses = cur.fetchall()
cur.close()

vornoiJson = json.loads(data[0][0])
transJson = json.loads(data[0][1])
houseJson = json.loads(houses[0][0])

# Step2. Data visualisation with ipyleaflet

### DB Usage
1. Most roads information can be obtained in table planet_osm_line
2. We use 'highway' field to filter different types of road.
3. Tags for 'highway':
   - Large road: 'residential'; Small road: 'footway'; Park road: 'service'; 'step' same as 'track'
   - The main tag we require by now is 'residential'


## Approximate a road by its starting and ending

In [24]:
conn = connect()
cur = conn.cursor()
### Main street from Garching
# cur.execute("SELECT ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM planet_osm_line "
#             "WHERE ST_Within(way,(SELECT way FROM planet_osm_polygon WHERE osm_id = -30971 )) = true "
#             "AND highway in ('residential');")# in('primary','secondary','primary_link');")

# Select houses and roads within vornoi polygon -> for each house, select the roads within specific radius
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(t1.way, 4326))), "
            "ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(ST_StartPoint(t2.way), 4326))))), "
            "ST_AsGeoJSON(ST_Collect((ST_FlipCoordinates(ST_Transform(ST_EndPoint(t2.way), 4326))))) "
            "FROM (SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false) AS t1, "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "
            "WHERE ST_DWithin(t1.way, t2.way, 60) = true GROUP BY t1.way;" % (vid,vid))

axpMap = cur.fetchall()
cur.close()

rdNodes = set()
for i in range(len(axpMap)):
    #house = json.loads(axpMap[i][0])['coordinates']
    start = json.loads(axpMap[i][1])['coordinates'][0]
    end = json.loads(axpMap[i][2])['coordinates'][0]
    #rdNodes.append(house)
    # Filter duplicated points
    rdNodes.add((start[0],start[1]))
    rdNodes.add((end[0],end[1]))

## Project houses only to the nearest road

In [107]:
# Select roads near one house
cur = conn.cursor()
# Create intermediate table
cur.execute("CREATE TABLE IF NOT EXISTS tempHsRd (hway geometry(Point,900913), rway geometry(LineString,900913), distance float);")
conn.commit()   
# clean entries
cur.execute("DELETE FROM temphsrd;")
conn.commit() 
cur.execute("INSERT INTO temphsrd "
            "(SELECT t1.way, t2.way, ST_Distance(t1.way, t2.way) "
            "FROM "
            "(SELECT way FROM vornoi_map WHERE vid = %d AND istransformer = false) AS t1 LEFT JOIN "
            "(SELECT way FROM planet_osm_line WHERE highway in ('residential') "
            "AND ST_DWithin(way,(SELECT way FROM vornoi WHERE vid = %d ), 1) = true) AS t2 "            
            "ON ST_DWithin(t1.way, t2.way, 100) = true);" % (vid,vid))
conn.commit() 

cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(hway, 4326))), "
            "ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_ClosestPoint(rway, hway), 4326))) "
            "FROM temphsrd "
            "WHERE (hway, distance) IN "
            "(SELECT hway, MIN(distance) FROM temphsrd GROUP BY hway);")
hsRdMap = cur.fetchall()
cur.close()

hsNodes = set()
for i in range(len(hsRdMap)):
#     house = json.loads(hsRdMap[i][0])['coordinates']
#     omk = Circle(location=house, weight=5, opacity = 0.7, color = 'red', radius = 2) 
#     tMap.add_layer(omk)

    proj = json.loads(hsRdMap[i][1])['coordinates']
    hsNodes.add((proj[0], proj[1]))

In [117]:
allNodes = rdNodes | hsNodes
tMap = cleanMap(transJson, vornoiJson)
for nd in allNodes:
    omk = Circle(location=nd, weight=5, opacity = 0.5, color = '#003300', radius = 2) 
    tMap.add_layer(omk)
tMap

## Make tables for nodes

In [118]:
conn = connect()
cur = conn.cursor()
tCenter = transJson['coordinates']
cur.execute("SELECT count(*) FROM approx_node;")
count = cur.fetchall()
if count[0][0] != 0:
    cur.execute("DELETE FROM approx_node WHERE ntype in (1,2,3);")
    
cur.execute("INSERT INTO approx_node(way, ntype) VALUES (ST_SetSRID(ST_Point(%f, %f),4326), 3);"
            %(tCenter[0], tCenter[1]))
for nd in rdNodes:
    cur.execute("INSERT INTO approx_node(way, ntype) VALUES (ST_SetSRID(ST_Point(%f, %f),4326), 1);"
                %(nd[0], nd[1]))
for nd in hsNodes:
    cur.execute("INSERT INTO approx_node(way, ntype) VALUES (ST_SetSRID(ST_Point(%f, %f),4326), 2);"
                %(nd[0], nd[1]))
conn.commit() 
cur.close()

## Minimum Spanning Tree Along Roads

In [125]:
cur = conn.cursor()
cur.execute("SELECT t1.nid, t2.nid, ST_Distance(ST_Transform(t1.way,26986), ST_Transform(t2.way,26986)) "
            "FROM (SELECT nid, way FROM approx_node) AS t1, "
            "(SELECT nid, way FROM approx_node) AS t2 "
            "WHERE t1.nid != t2.nid ;")
edgeData = cur.fetchall()
graph = []
for ele in edgeData:
    graph.append("%d %d %f" % ele)
G = nx.parse_edgelist(graph, nodetype = int, data=(('weight',float),))
MST = nx.minimum_spanning_tree(G)
print approx.node_connectivity(G), len(MST.nodes())

edgeStr = str(MST.edges()).replace("[", "(").replace("]",")")
cur.execute("SELECT ST_AsGeoJSON(ST_MakeLine(t1.way,t2.way))"
            "FROM (SELECT nid, way FROM approx_node) AS t1, "
            "(SELECT nid, way FROM approx_node) AS t2 "
            "WHERE (t1.nid,t2.nid) in %s ;"%edgeStr)
lines = cur.fetchall()
cur.close()

tMap = cleanMap(transJson, vornoiJson)
for line in lines:
    lineJson = json.loads(line[0])['coordinates']   
    pl = Polyline(locations=lineJson, weight=2, color='red', opacity=0.5)
    tMap.add_layer(pl)
tMap

97 98
